In [158]:
import tabula
from pypdf import PdfReader
import re
import pandas as pd
import numpy as np

In [274]:
# 목차에서 페이지 번호를 찾아 찾았으면 시작 페이지번호를, 못 찾았으면 -1을 리턴
def title_to_page_number(reader, chapter_title):
    length_of_table_of_contents = get_first_page_index(reader)
    for idx in range(length_of_table_of_contents):
        page = reader.pages[idx]
        table_of_contents = page.extract_text()
        start_index = table_of_contents.find(chapter_title)
        if start_index != -1:
            end_of_line = table_of_contents[start_index:].find("\n") + start_index
            blank = table_of_contents[start_index:end_of_line].rfind(" ") + start_index
            return int(table_of_contents[blank+1:end_of_line])
    return -1

# 시작 페이지 인덱스를 리턴
def title_to_page_index(reader, chapter_title):
    page_number = title_to_page_number(reader, chapter_title)
    if page_number == -1:
        return -1
    else:
        return page_number + get_first_page_index(reader)

# 첫 페이지의 페이지 번호를 리턴
def get_first_page_index(reader):
    for idx, page in enumerate(reader.pages):
        text = page.extract_text()
        if text.rfind("Page 1") != -1:
            return idx

In [310]:
reader = PdfReader("data/hyundai.pdf")
#number_of_pages = len(reader.pages)
#obj = page.get_object()
start_page = title_to_page_index(reader, "연결 손익계산서")
page = reader.pages[start_page]
text = page.extract_text()
page = reader.pages[start_page + 1]
text = "".join([text, page.extract_text()])

In [312]:
text

'2-2. 연결 손익계산서 \n연결 손익계산서\n제 56 기 2023.01.01 부터 2023.12.31 까지\n제 55 기 2022.01.01 부터 2022.12.31 까지\n제 54 기 2021.01.01 부터 2021.12.31 까지\n(단위 : 백만원)\n\u3000 제 56 기 제 55 기 제 54 기\n매출액 162,663,579 142,151,469 116,448,159\n매출원가 129,179,183 113,879,569 94,721,313\n매출총이익 33,484,396 28,271,900 21,726,846\n판매비와관리비 18,357,495 18,446,972 15,200,325\n영업이익 15,126,901 9,824,928 6,526,521\n공동기업및관계기업투\n자손익2,470,933 1,557,630 1,302,150\n금융수익 1,559,538 985,893 841,196\n금융비용 970,700 879,638 512,836\n기타수익 1,782,333 1,930,914 1,377,997\n기타비용 2,350,343 2,238,256 1,745,985\n법인세비용차감전순이\n익17,618,662 11,181,471 7,789,043\n법인세비용(수익) 4,626,640 2,979,168 2,266,823\n계속영업연결당기순이\n익12,992,022 8,202,303 5,522,220\n중단영업당기순이익(손\n실)(719,721) (218,689) 170,857\n연결당기순이익 12,272,301 7,983,614 5,693,077\n당기순이익(손실)의 귀\n속\u3000 \u3000 \u3000\n\u3000지배기업소유주지분 11,961,717 7,364,364 4,942,356\n\u3000비지배지분 310,584 619,250 750,721\n주당이익 \u3000 \u3000 \u3000\n\u3000기본주당이익 (단위 :\n원)\u3000 \u3000 \u3000\n\u3000\u3000보통주기본주당이\n익(손실) (단위 : 

In [325]:
r = re.compile("(?<!\. )연결 포괄손익계산서")
print(r.search(text))

<re.Match object; span=(1437, 1447), match='연결 포괄손익계산서'>


In [299]:
r = re.compile(r"(제 [0-9]+ 기 [0-9]{4}[.][0-9]{2}[.][0-9]{2} 부터 [0-9]{4}[.][0-9]{2}[.][0-9]{2} 까지\n){3}")
r.search(text)


<re.Match object; span=(424, 529), match='제 56 기 2023.01.01 부터 2023.12.31 까지\n제 55 기 2022.0>

In [ ]:
index_of_period = r.search(text).start()
text[text[:index_of_period-1].rfind("\n")+1:index_of_period-1]

In [238]:
r = re.compile(r"\u3000 (제 [0-9]+ 기){3}\n")

In [223]:
# 삼성 : 80~83
# 현대 : 69~73
# 한전 : 165~167

tabula_result_dfs = tabula.read_pdf(
    "data/hyundai.pdf",
	pages="69-73",
	stream=True,
    lattice=True
)
print(f"Data Type :{type(tabula_result_dfs)}")
print(f"Data Length: {len(tabula_result_dfs)}")
for index, table in enumerate(tabula_result_dfs):
    print(f"\nData Index: {index}")
    print(type(table))
    print(table.head())

Data Type :<class 'list'>
Data Length: 7

Data Index: 0
<class 'pandas.core.frame.DataFrame'>
  Unnamed: 0       제 56 기      제 55 기      제 54 기
0         자산          NaN         NaN         NaN
1       유동자산  101,724,717  96,389,273  88,565,366
2   현금및현금성자산   19,166,619  20,864,879  12,795,554
3     단기금융상품    7,339,968   5,774,597   6,949,333
4     기타금융자산    2,802,611   5,934,745  12,396,646

Data Index: 1
<class 'pandas.core.frame.DataFrame'>
        부채  Unnamed: 0  Unnamed: 1  Unnamed: 2
0     유동부채  73,362,103  74,236,472  64,236,787
1     매입채무  10,952,046  10,797,065   9,155,255
2     미지급금   8,642,808   8,277,891   6,335,645
3    단기차입금   9,035,548  11,366,480  13,087,836
4  유동성장기부채  25,109,158  25,574,131  20,578,902

Data Index: 2
<class 'pandas.core.frame.DataFrame'>
  Unnamed: 0       제 56 기       제 55 기       제 54 기
0        매출액  162,663,579  142,151,469  116,448,159
1       매출원가  129,179,183  113,879,569   94,721,313
2      매출총이익   33,484,396   28,271,900   21,726,846
3    판매비와관

c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [224]:
year_regular_expr = re.compile("제 [0-9]+ 기")
columns = []
dfs = []
for tabula_result_df in tabula_result_dfs[0:6]:
    tabula_result_df_columns = [item for item in tabula_result_df.columns]
    if year_regular_expr.match(tabula_result_df_columns[-1]):
        columns = tabula_result_df_columns
        dfs.append(tabula_result_df)
    else:
        temp_df = pd.DataFrame(tabula_result_df_columns, index = columns).T
        dfs[-1] = pd.concat([dfs[-1], temp_df], ignore_index=True)
        tabula_result_df.columns = columns
        dfs[-1] = pd.concat([dfs[-1], tabula_result_df], ignore_index=True)
for df in dfs:
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x : x.replace("\r", "") if x is not np.nan else x)

In [227]:
dfs[1]

,Unnamed: 0,제 56 기,제 55 기,제 54 기
0,매출액,"162,663,579","142,151,469","116,448,159"
1,매출원가,"129,179,183","113,879,569","94,721,313"
2,매출총이익,"33,484,396","28,271,900","21,726,846"
3,판매비와관리비,"18,357,495","18,446,972","15,200,325"
4,영업이익,"15,126,901","9,824,928","6,526,521"
5,공동기업및관계기업투자손익,"2,470,933","1,557,630","1,302,150"
6,금융수익,"1,559,538","985,893","841,196"
7,금융비용,"970,700","879,638","512,836"
8,기타수익,"1,782,333","1,930,914","1,377,997"
9,기타비용,"2,350,343","2,238,256","1,745,985"


In [117]:
""" ㅋㅋㅋ lattice = True 주니까 이거 할 필요 없어졌음
title_string = ""
edge_thickness = 0
edge_ascending = True
new_df = []
new_row = []
for idx, row in dfs[1].iterrows():
    print(idx)
    row_values = dfs[1].iloc[idx, 1:]
    row_title = dfs[1].iat[idx, 0]
    if row_values.isnull().sum() == len(row_values):
        if edge_ascending == True:
            edge_thickness += 1
            title_string = "".join([title_string, row_title])
            print(f"edge_ascending : {edge_ascending}, edge_thickness : {edge_thickness}, title_string : {title_string}")
        else:
            edge_thickness -= 1
            title_string = "".join([title_string, row_title])
            print(f"edge_ascending : {edge_ascending}, edge_thickness : {edge_thickness}, title_string : {title_string}")
        #print(dfs[1].iat[idx,0], "All Null")
    else:                                   # Null이 아닌 값이 채워져 있으면
        if edge_thickness > 0:
            edge_ascending = False
        title_string = "".join([title_string, row_title])
        contents = row_values
    if edge_thickness == 0:
        new_row.append(title_string)
        new_row.extend(contents.values)
        new_row = pd.Series(new_row, index = row.index)
        new_df.append(new_row)
        new_row = []
        edge_ascending = True
        title_string = ""
        #print(dfs[1].iat[idx,0], "values")
"""

0
1
2
edge_ascending : True, edge_thickness : 1, title_string : 후속적으로 당기손익
3
4
edge_ascending : False, edge_thickness : 0, title_string : 후속적으로 당기손익으로 재분류되지 않는항목
5
edge_ascending : True, edge_thickness : 1, title_string : 기타포괄손익-공정
6
7
edge_ascending : False, edge_thickness : 0, title_string : 기타포괄손익-공정가치측정금융자산관련손익
8
edge_ascending : True, edge_thickness : 1, title_string : 확정급여제도의 재
9


TypeError: sequence item 1: expected str instance, float found

#### trim돼서 계층을 알 수 없으니, pypdf로 나온 값을 참고해서, 맨 왼쪽꺼는 \n 다음에 \u3000이 몇번 나왔는지로 계층을 판단하자.
#### 그리고 현대꺼 줄바꿈 주의..